Берём строку без пробелов и пробуем на каждом её начале понять, что это за кусок: целое число, английское слово, или русское слово из словаря pymorphy2.

Из всех вариантов разбиения выбираем тот, где сумма квадратов длин кусочков максимальна, то есть длинные осмысленные куски.

Где добавляем границы:
- Между кириллицей и латиницей

- Между буквой и цифрой и наоборот

- Между строчной и заглавной

Еще я учитываю базовые правила пунктуации: не ставлю пробел перед , . ! ? : ; … % ) ] } » и после ( [ { «. 
В остальных местах пробел ставится после знака, а не до него.

In [23]:
!pip install pymorphy2 pymorphy2-dicts-ru

In [24]:
import re
from functools import lru_cache
import inspect
from collections import namedtuple

if not hasattr(inspect, "getargspec"):
    """ Просто имитация inspect.getargspec, чтобы не ругался pymorphy2"""
    FullArgSpec = inspect.getfullargspec
    def getargspec(func):
        fs = FullArgSpec(func)
        ArgSpec = namedtuple("ArgSpec", "args varargs keywords defaults")
        return ArgSpec(fs.args, fs.varargs, fs.varkw, fs.defaults)
    inspect.getargspec = getargspec

In [25]:
import pymorphy2

# Регулярки для определения киррилических и латинских букв
_cyr_pat = re.compile(r"^[А-Яа-яЁё]+$")
_lat_pat = re.compile(r"^[A-Za-z]+$")

def is_cyrillic(s: str) -> bool:
    """True, если строка состоит только из кириллических букв"""
    return bool(_cyr_pat.fullmatch(s))

def is_latin(s: str) -> bool:
    """True, если строка состоит только из латинских букв"""
    return bool(_lat_pat.fullmatch(s))

In [26]:
# Разделители внутри числа и знаки препинания (минус, тире, дефис, так как тире знак препинания, а дефис может быть частью слова нет)
NUM_SEPS = {",", ".", "'", "’", "\u202F", "\u00A0", "\u2009"}
MINUS_SIGNS = {"-", "−"}

def check_number(s: str, start: int = 0) -> int:
    """
    Возвращаем индекс конца числа, начиная с start. 
    Число может содержать разделители из NUM_SEPS 
    и начинаться с минуса из MINUS_SIGNS.
    Если числа нет, возвращаем start.
    """
    i = start
    if i < len(s) and (s[i] in MINUS_SIGNS or s[i] == "+"):
        i += 1
    if i >= len(s) or not s[i].isdigit():
        return start
    last_separated = False
    while i < len(s):
        ch = s[i]
        if ch.isdigit():
            last_separated = False
            i += 1
        elif ch in NUM_SEPS:
            if last_separated: # подряд не может быть два разделителя
                break
            last_separated = True
            i += 1
        else:
            break
    # Число не должно заканчиваться на разделитель
    while i > start and not s[i - 1].isdigit():
        i -= 1
    return i

def check_latin(s: str, start: int = 0) -> int:
    """Идём вперёд, пока буквы латинские 
    и возвращаем индекс первого не-латинского символа."""
    i = start
    while i < len(s) and s[i].isalpha() and is_latin(s[i]):
        i += 1
    return i



In [27]:
# Перед этими символами не должно быть пробела
PUNCT = set(",.;:!?…%)]}»")
B_PUNCT  = set("([«")

class NorvigSegmenter:
    """
    Пытаемся разложить вход на осмысленные токены (числа, латинские слова, русские слова из pymorphy2) так, 
    чтобы максимизировать сумму квадратов длин токенов. Кэшируем, чтобы быстрее было.
    """
    def __init__(self, max_word_len: int = 32):
        # Инициализируем морф-анализатора и доступа к DAWG слов
        self.morph = pymorphy2.MorphAnalyzer()
        self.words_dawg = self.morph.dictionary.words
        self.max_word_len = max_word_len
        try:
            self.segment_recursive.cache_clear()  # очищаем кэш, если он был
        except Exception:
            pass

    @lru_cache(maxsize=None)
    def segment_recursive(self, text: str):
        """
        Рекурсивно бьёт text на токены и максимизирует сумму квадратов длин токенов.
        Возвращает (score, tokens).
        """
        if not text:
            return 0.0, []

        candidates = []

        # 1) Если начинается с цифры — берем число полностью
        if text[0].isdigit() or (text[0] in MINUS_SIGNS or text[0] == "+"):
            j = check_number(text, 0)
            if j > 0:
                tok = text[:j]
                score_rest, rest = self.segment_recursive(text[j:])
                candidates.append((len(tok) ** 2 + score_rest, [tok] + rest))

        # 2) Если начинается с латинской буквы — берем слово полностью
        if text[0].isalpha() and is_latin(text[0]):
            j = check_latin(text, 0)
            if j > 0:
                tok = text[:j]
                score_rest, rest = self.segment_recursive(text[j:])
                candidates.append((len(tok) ** 2 + score_rest, [tok] + rest))

        # 3) Ищем русские слова в DAWG
        max_len = min(len(text), self.max_word_len)
        for i in range(1, max_len + 1):
            w = text[:i]
            if w in self.words_dawg:
                score_rest, rest = self.segment_recursive(text[i:])
                candidates.append((len(w) ** 2 + score_rest, [w] + rest))

        # 4) Если ничего не подошло — берём первый символ как отдельный токен
        if not candidates:
            score_rest, rest = self.segment_recursive(text[1:])
            return score_rest, [text[0]] + rest

        return max(candidates)

    def segment(self, text: str):
        """
        Возвращает множество позиций-границ, где нужно ставить пробелы.
        """
        text = text.lower()
        _, words = self.segment_recursive(text)

        # Склеиваем подряд идущие одиночные кириллические буквы в слова
        final_tokens, buf = [], ""
        for w in words:
            if len(w) == 1 and is_cyrillic(w):
                buf += w
            else:
                if buf:
                    final_tokens.append(buf)
                    buf = ""
                final_tokens.append(w)
        if buf:
            final_tokens.append(buf)

        # Определяем границы между токенами
        positions = set()
        cur = 0
        for i in range(len(final_tokens) - 1):
            left = final_tokens[i]
            right = final_tokens[i + 1]
            lch, rch = left[-1], right[0]

            if rch in PUNCT:
                pass
            elif lch in B_PUNCT:
                pass
            elif (len(left) == 1 and not left.isalnum() and
                  len(right) == 1 and not right.isalnum()):
                pass
            else:
                positions.add(cur + len(left))

            cur += len(left)
        return positions

In [28]:
def predict_spaces(text: str, segmenter: NorvigSegmenter) -> str:
    """
    Возвращает строку вида "[i, j, ...]" — индексы пробелов.
    """
    if not text:
        return "[]"
    
    algo_pos = segmenter.segment(text)

    # Эвристики: кириллица↔латиница, буква↔цифра, цифра↔буква, lower→Upper
    heur_pos = set()
    for i in range(len(text) - 1):
        c, n = text[i], text[i + 1]
        is_cyr_c = is_cyrillic(c.lower())
        is_cyr_n = is_cyrillic(n.lower())
        is_lat_c = is_latin(c.lower())
        is_lat_n = is_latin(n.lower())

        if (
            (c.isalpha() and n.isalpha() and ((is_cyr_c and is_lat_n) or (is_lat_c and is_cyr_n))) or
            (c.isalpha() and n.isdigit()) or
            (c.isdigit() and n.isalpha()) or
            (c.islower() and n.isupper())
        ):
            heur_pos.add(i + 1)

    final_pos = sorted(algo_pos.union(heur_pos))
    return "[]" if not final_pos else f"[{', '.join(map(str, final_pos))}]"

if __name__ == "__main__":
    seg = NorvigSegmenter()

    samples = [
        "куплюдом",
        "Продамкрышудёшево123",
        "самыйniceфенDyson"
    ]
    for s in samples:
        print(f"{s!r} -> {predict_spaces(s, seg)}")


'куплюдом' -> [5]
'Продамкрышудёшево123' -> [6, 11, 17]
'самыйniceфенDyson' -> [5, 9, 12]


In [30]:
import pandas as pd
from pathlib import Path

INPUT_PATH  = "dataset_1937770_3.txt" 
OUTPUT_PATH = "submission.csv"

seg = NorvigSegmenter()
p = Path(INPUT_PATH)

if p.suffix.lower() == ".csv":
    df = pd.read_csv(p, encoding="utf-8")
else:
    rows = [line.rstrip("\n").split(",", 1) for line in p.open("r", encoding="utf-8")]
    if rows and not rows[0][0].isdigit():
        rows = rows[1:]
    df = pd.DataFrame(rows, columns=["id", "text_no_spaces"])
    df["id"] = df["id"].astype(int)

# нормализуем имена
if "text_no_spaces" not in df.columns and "text" in df.columns:
    df = df.rename(columns={"text": "text_no_spaces"})
if "id" not in df.columns:
    df.insert(0, "id", range(len(df)))

df["predicted_positions"] = df["text_no_spaces"].apply(lambda s: predict_spaces(s, seg))

out = df[["id", "text_no_spaces", "predicted_positions"]]
out.to_csv(OUTPUT_PATH, index=False, encoding="utf-8")

print("[cell5] head:\n", out.head())


[cell5] head:
    id                 text_no_spaces  predicted_positions
0   0                куплюайфон14про          [5, 10, 12]
1   1             ищудомвПодмосковье            [3, 6, 7]
2   2  сдаюквартирусмебельюитехникой  [4, 12, 13, 20, 21]
3   3     новыйдивандоставканедорого          [5, 10, 18]
4   4                отдамдаромкошку              [5, 10]
